Data Prep and Imports

In [1]:
import pandas as pd
import numpy as np

leak_data_raw =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
leak_data_raw.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]

weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")
columns_to_average = ['Barometric_Pressure', 'Humidity', 'Temperature', 'Wind_Direction', 'Wind_Speed']
for col in columns_to_average:
    weather_df[col] = weather_df.groupby('timestamp')[col].transform('mean')
weather_df = weather_df.drop_duplicates()
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9
weather_df = weather_df.rename(columns={'timestamp': 'tTime'})

leak_rate_df = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")
leak_rate_df =  leak_rate_df[['LeakRate', 'tStart', 'tEnd', 'Duration']]
leak_rate_df['LeakRate'] = leak_rate_df.groupby('tStart')['LeakRate'].transform('mean')
leak_rate_df = leak_rate_df.drop_duplicates()


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_61668/2550901138.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9


In [2]:
leak_data_raw['sensor_avg'] = 0
for i in range(len(leak_data_raw)):
    leak_data_raw.iloc[i,-1] =  leak_data_raw.iloc[i, 2:-1].mean()


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_61668/4017535932.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1050.1875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  leak_data_raw.iloc[i,-1] =  leak_data_raw.iloc[i, 2:-1].mean()


Merging Data Sets

In [3]:
# # Leak and weather data merge
# leak_data_raw['tTime'] = leak_data_raw['Time']
# leak_data_raw['tTime'] = pd.to_datetime(leak_data_raw['tTime'], unit='s')
# weather_df['tTime'] = pd.to_datetime(weather_df['tTime'], unit='s')

# leak_data_raw['tTime'] = leak_data_raw['tTime'].dt.floor('min').dt.round('min')
# df = pd.merge(leak_data_raw, weather_df, on='tTime', how='left')
# df = df.drop(['temp', 'tTime'], axis=1)

In [4]:
# Normalizing leak intensity
# df['leak_amount'] = 0
# for index, row in leak_rate_df.iterrows():
#     in_range = df['Time'].between(row['tStart'], row['tEnd'], inclusive='both')
#     df.loc[in_range, 'leak_amount'] = np.array((row['LeakRate'] * (row['Duration'])), dtype=np.int32)
# df['leak_amount_norm'] = (df['leak_amount'] - df['leak_amount'].min()) / (df['leak_amount'].max() - df['leak_amount'].min())


# # Final prep
# df = df.drop(['leak_amount'],axis=1)
# df = df.astype(dtype='float32')


In [5]:
df = leak_data_raw.drop(['temp','Time'], axis=1)
# df = df.astype(dtype='float32')


In [6]:
df

,Sensor 1,Sensor 2,Sensor 3,Sensor 4,Sensor 5,Sensor 6,Sensor 7,Sensor 8,Sensor 9,Sensor 10,...,Sensor 16,Sensor 17,Sensor 18,Sensor 19,Sensor 20,Sensor 21,Sensor 22,Sensor 23,Sensor 24,sensor_avg
0,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,1082.0,1037.0,...,735.0,1088.0,991.5,931.0,1040.0,1040.0,1111.0,1080.0,1089.0,1050.187500
1,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,1085.0,1039.0,...,765.0,1090.0,984.0,928.5,1043.0,1052.0,1105.5,1075.0,1095.0,1049.416667
2,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,1088.0,1053.0,...,762.0,1095.0,993.0,926.0,1039.0,1033.0,1100.0,1021.0,1093.0,1045.083333
3,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,1091.0,1057.0,...,762.0,1100.0,999.0,929.0,1047.5,1014.0,1107.0,1059.5,1110.0,1048.854167
4,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,1090.5,1061.0,...,763.0,1100.0,1004.5,944.0,1056.0,1019.0,1098.0,1098.0,1107.5,1049.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83408,853.0,901.0,990.0,813.0,791.0,911.0,887.0,943.0,968.0,966.0,...,661.0,1037.0,900.5,795.0,1007.0,955.0,1020.0,949.0,987.0,920.666667
83409,852.5,899.0,1007.0,801.0,788.5,900.0,890.0,931.0,984.0,975.0,...,644.0,1056.0,901.0,799.5,1009.0,954.0,1018.0,955.0,986.0,917.687500
83410,852.0,901.0,1011.0,811.0,786.0,909.0,899.0,945.0,967.0,996.0,...,643.0,1059.0,908.0,804.0,1016.0,961.0,1017.0,938.0,982.0,920.166667
83411,859.0,912.0,1010.0,811.0,764.0,909.0,892.0,959.0,969.0,945.0,...,609.0,1071.0,897.0,802.0,1019.0,963.0,1026.0,948.0,990.0,917.437500


Train Model

In [27]:
# Imports
import torch

import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import model as m


model = m.LightningLSTM()

In [8]:
sensor1_df_normalized = (df['Sensor 1'] - df['Sensor 1'].min()) / (df['Sensor 1'].max() - df['Sensor 1'].min())
sensor1_df_normalized = sensor1_df_normalized.astype(dtype='float32')

In [9]:
sensor1_df_normalized.iloc[0:60, ]

0     0.315134
1     0.318008
2     0.307471
3     0.318966
4     0.322318
5     0.325671
6     0.347701
7     0.342433
8     0.337165
9     0.321839
10    0.330460
11    0.330460
12    0.330460
13    0.311303
14    0.301245
15    0.291188
16    0.267241
17    0.278257
18    0.289272
19    0.285441
20    0.281609
21    0.267241
22    0.268199
23    0.274904
24    0.281609
25    0.288314
26    0.295019
27    0.301724
28    0.297893
29    0.326628
30    0.329502
31    0.332375
32    0.338123
33    0.338123
34    0.319923
35    0.306513
36    0.293103
37    0.289272
38    0.280651
39    0.280651
40    0.280651
41    0.294061
42    0.297893
43    0.301724
44    0.318966
45    0.313218
46    0.307471
47    0.295977
48    0.274425
49    0.252874
50    0.226054
51    0.225096
52    0.222222
53    0.219349
54    0.219828
55    0.220307
56    0.222222
57    0.230843
58    0.232759
59    0.234674
Name: Sensor 1, dtype: float32

In [10]:
inputs = []
labels = []
for i in range(60, len(sensor1_df_normalized)):
    inputs.append(sensor1_df_normalized.iloc[i-60:i, ])
    labels.append(sensor1_df_normalized.iloc[i,])
inputs
inputs = torch.tensor(np.array(inputs)) # convert to numpy
labels = torch.tensor(np.array(labels))

dataset =  TensorDataset(inputs, labels) #fix dimension
dataloader = DataLoader(dataset)



In [11]:
trainer = L.Trainer(max_epochs=5)
trainer.fit(model,train_dataloaders=dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
/opt/homebrew/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 4: 100%|██████████| 83353/83353 [08:56<00:00, 155.48it/s, v_num=23]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 83353/83353 [08:56<00:00, 155.48it/s, v_num=23]


In [59]:
test_data = pd.read_csv("Gasleak Data Sets/validation_files/sensor_readings.csv")
test_data.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]
sensor1_test_normalized = (test_data['Sensor 1'] - test_data['Sensor 1'].min()) / (test_data['Sensor 1'].max() - test_data['Sensor 1'].min())

In [100]:
# Set the window size
window_size = 60

# Create a list of input sequences
inputs = [sensor1_test_normalized.iloc[i:i+window_size].values for i in range(len(sensor1_test_normalized) - window_size)]

# Create a list of corresponding labels
labels = sensor1_test_normalized.iloc[window_size:].values

# # Convert the lists to pandas objects
# test_inputs = pd.DataFrame(inputs)
# test_labels = pd.Series(labels)

test_inputs = torch.tensor(np.array(inputs))
test_labels = torch.tensor(np.array(labels))


# prediction_df = pd.concat([test_inputs, test_labels], axis=1)

In [102]:
# def apply_model(x):
#     model

# prediction_df['prediction_normalized'] =  test_inputs.apply(lambda x: model(x).detach(), axis=0)
# prediction_df['prediction_normalized'] = model(test_inputs[0]).detach()
# prediction = prediction_normalized * (test_data['Sensor 1'].max() - test_data['Sensor 1'].min()) + test_data['Sensor 1'].min()

# for i in range(len(test_inputs)):
prediction = model(test_inputs[0]).detach()



RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [14]:
# path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
# trainer = L.Trainer(max_epochs=4)
# trainer.fit(model,train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

In [16]:
# print(model(torch.tensor([848.0,850.0,855.0,854.0,855.0,853.0,852.5,852.0,859.0,855.0])).detach())

tensor([0.])
